# <span style="color: steelblue;">Differential Gene expression(DGE) analysis using scaLR </span>

## <span style="color: steelblue;"> Keypoints </span>
- Differential gene expression (DGE) analysis is a method employed in genomics to evaluate and compare gene expression levels across different sample groups. This can include comparisons between healthy and diseased tissues or cells subjected to various treatments.

- In single-cell RNA-seq (scRNA-seq) data, the gene expression matrix is organized into various hierarchical groups. These groups may include regions from which samples were collected, clinical conditions such as different disease stages or normal states. For each individual subject in the study, RNA-seq generates a number of cells, with gene expression levels varying across different cell types.

- Major single-cell differential gene expression (scDGE) analysis approaches are : Pseudobulk, Mixed-effects models, and Differential distribution based methods. This tutorial explains how to perform scDGE analysis using scaLR DGE module which includes: ```Pseudobulk``` and ```Linear mixed-effects model```approaches.

## <span style="color: steelblue;">Imports</span>

In [ ]:
import sys
sys.path.append('/path/to/scaLR/')

In [ ]:
import os
import pandas as pd
import scanpy as sc
from matplotlib_venn import venn2
import numpy as np
from scalr.analysis import dge_lmem
from scalr.analysis import dge_pseudobulk
%reload_ext autoreload
%autoreload 2

## <span style="color: steelblue;">Loading of data and its exploration</span>

In [ ]:
dirpath = '/path_to_scaLR/tutorials/analysis/differential_gene_expression/'
adata = sc.read_h5ad(os.path.join(dirpath,'test_data.h5ad'),backed='r')

In [ ]:
#Structure of the anndata
adata

In [ ]:
#Cell barcodes and metadata
adata.obs.head()

In [ ]:
# Gene information metadata
adata.var.head()

In [ ]:
#Information about subjects
adata.obs.donor_id.unique()

In [ ]:
#Celltype information
adata.obs.cell_type.value_counts()

In [ ]:
#Clinical conditions 
adata.obs.disease.value_counts()

## <span style="color: steelblue;">DGE analysis using Pseudobulk approach</span>

- ```Pseudobulk``` approach involves aggregating the unique molecular identifier (UMI) counts within each cell type of all
  subject, effectively creating a "pseudobulk" dataset.
- In this method, the scRNA-seq data is summarized to resemble bulk RNA-seq data, where the UMI counts from all cells
  of a specific type are combined for each subject.
- This aggregated dataset can then be analyzed using DGE methods designed for bulk RNA-seq by treating each
  cell type's UMI counts as if they were from a bulk sample.

### <span style="color: steelblue;">Analysis</span>

- The analysis starts by selecting a subset of ```cell types``` for the differential gene expression (DGE) analysis, followed by
  extracting individual subsets of the `AnnData` object for each cell type.
- The ```donor_id``` in ```adata.obs``` will be used to identify the subjects, while the ```disease``` column will serve as the
  ```design_factor```, with `['COVID-19', 'normal']` representing the two factor levels for the ```design_factor```.
- The required and optionalparameters are outlined below.

In [ ]:
dge_pbk = dge_pseudobulk.DgePseudoBulk(celltype_column = 'cell_type',
                                       design_factor = 'disease',
                                       factor_categories = ['COVID-19', 'normal'],
                                       sum_column = 'donor_id',
                                       cell_subsets = ['non-classical monocyte','natural killer cell'],
                                       min_cell_threshold = 1,
                                       fold_change = 1.5,
                                       p_val = 0.05,
                                       y_lim_tuple = None,
                                       save_plot = True,
                                       logger= 'FlowLogger')

#### <span style="color: steelblue;">Parameters </span>
- (* marked are required)
   
    - ```*celltype_column``` : Column name in `anndata.obs` containing all the cell types
    - ```*design_factor``` : Column name in `anndata.obs` containing different factor levels or categories for differential gene expression analysis           
    - ```*factor_categories``` : List of conditions in ```design_factor``` used to create the design matrix, with the last category in the list serving as the reference to the first. For example, in ```['disease_1','normal']```, ```normal``` serves as the reference.
    - ```*sum_column``` : Column name to sum values across samples.  
    - ```*cell_subsets``` : Selcted list of cell types in ```celltype_column``` to subset the anndata.
    - ```min_cell_threshold``` : Minimum number of subjects with aggregated nonzero gene expression values.
      Each subject has the aggregated expression value for each gene of the selected cell type.
      If `min_cell_threshold = 1` is specified, genes will be filtered out unless they have at least one non-zero value in the subjects.
    - ```fold_change``` : Fold change to filter the differentially expressed genes for volcano plot.
    - ```p_val``` : ```p```value to filter differentially expressed genes for volcano plot.
    - ```y_lim_tuple``` : Values to adjust the Y-axis limits of the plot.
    - ```save_plot``` : Boolean value to save the plot.
    - ```logger``` : For status logging

In [ ]:
#Generate analysis and save the result
dge_pbk.generate_analysis(adata,dirpath)

### <span style="color: steelblue;">Result</span>

In [ ]:
# Pseudobulk(pbk)
pbk_result_ncm = pd.read_csv(f'{dirpath}/pseudobulk_dge_result/pbkDGE_non-classicalmonocyte_COVID-19_vs_normal.csv')
pbk_result_ncm.head()

- DGE result for each gene in ```non-classicalmonocyte```
    - ```gene``` - Gene name
    -  ```baseMean``` - Mean gene expression
    - ```log2FoldChange``` : log2 fold change in gene expression in ```COVID-19``` compared to ```normal``` subjects
    - ```lfcSE``` : Standard Error for log2 fold change
    - ```stat``` : ```Wald's test``` statistics
    - ```pvalue``` : ```p``` value
    - ```padj``` : Adjusted ```p``` value
  
- Volcano plots of ```log2FoldChange``` vs ```-log10(pvalue)```

    ![Alt text](pbkDGE_non-classicalmonocyte_COVID-19_vs_normal.svg)

## <span style="color: steelblue;">DGE analysis using Linear Mixed Effects Model (LMEM) approach </span>

- A linear mixed-effects model can incorporate both fixed and random effects. Fixed effects remain consistent across the population, while random effects vary across different groups or levels within the data and are modeled as random variables with their own distribution, typically assumed to follow a normal distribution.
- In scRNA-seq data, these effects allow the model to treat certain parameters as random variables at the subject level, while fixing others at a higher level, such as the population mean for a clinical condition.
- In the same dataset, we will explore both fixed and random effects parameters and conduct differential gene expression analysis, taking these effects into account.

### <span style="color: steelblue;">Analysis</span>

- We'll select a subset of cell types for the differential gene expression (DGE) analysis. 
- Next, we'll subset the ```AnnData``` object for each cell type individually. 
- The final linear mixed-effects model (LMEM) analysis will use the ```disease``` column as the ```fixed effect``` parameter and ```donor_id``` as the ```random effect``` or ```group``` parameter. The required and optional parameters are listed below.

In [ ]:
dge_lm = dge_lmem.DgeLMEM(fixed_effect_column = 'disease',
                          fixed_effect_factors = ['COVID-19', 'normal'],
                          group = 'donor_id',
                          celltype_column = 'cell_type',
                          cell_subsets = ['non-classical monocyte', 'natural killer cell'],
                          min_cell_threshold = 10,
                          n_cpu = 4,
                          gene_batch_size = 1000,
                          coef_threshold = 0,
                          p_val = 0.05,
                          save_plot = True,
                          logger= 'FlowLogger')

#### <span style="color: steelblue;">Parameters </span>
- (* marked are required)
   
    - *```fixed_effect_column``` : Column name in ```anndata.obs``` containing different factor levels or categories for
      differential gene expression analysis. This serves as the ```fixed_effect``` parameter.           
    - *```fixed_effect_factors``` : List of conditions in ```fixed_effect_column``` used to create the design matrix, with the last
      category in the list serving as the reference to others. For example, in ```['disease_1','disease_2','normal']```,
      ```normal``` serves as the reference.
    - *```group``` : Column name to act as a ```random_effect``` parameter for mixed effect model.
    - *```celltype_column``` : Column name in ```anndata.obs``` containing all the cell types. Analysis can be done without this
      parameter, i.e. without susetting the input data as per the celltypes, but it is better to fix the data with a particular
      celltype to remove the cell specific confounding effects.  
    - *```cell_subsets``` : Selcted list of cell types in ```celltype_column``` to subset the anndata.
    - ```min_cell_threshold``` : Minimum number of cells with nonzero values for a gene.
    - ```n_cpu``` : Number of CPUs for parallelization.
    - ```gene_batch_size``` : Number of genes in a batch of processing.
    - ```coef_threshold``` : Threshold to filter up and down regulated genes in volcano plot.
    - ```p_val``` : ```p```value to filter differentially expressed genes for volcano plot.
    - ```y_lim_tuple``` : Values to adjust the Y-axis limits of the plot.
    - ```save_plot``` : Boolean value to save the plot.
    - ```logger``` : For status logging

In [ ]:
#Generate analysis and save the result
dge_lm.generate_analysis(adata,dirpath)

### <span style="color: steelblue;">Result</span>

In [ ]:
# LMEM(lmem)
lmem_result_ncm = pd.read_csv(f'{dirpath}/lmem_dge_result/lmem_DGE_non-classicalmonocyte.csv')
lmem_result_ncm.head()

- Model results for each gene in ```non-classicalmonocyte```
    - ```gene``` - Gene name
    - ```coef_COVID-19``` : Coefficient difference of ```COVID-19``` subjects compared to ```normal``` ones
    - ```SEcoef_COVID-19``` : Coefficient Standard Error
    - ```pval_COVID-19``` : ```p``` value
    - ```stat_COVID-19``` : ```Wald's test``` statistics
    - ```adj_pval_COVID-19``` : Adjusted ```p``` value

- Volcano plot of ```coef_COVID-19``` vs ```-log10(pval_COVID-19)```

  ![Alt text](lmem_DGE_non-classicalmonocyte_COVID-19.svg)
  

## <span style="color: steelblue;">DGE analysis for large Anndata with Nohang Up</span>

- For running the ```DgePseudoBulk/DgeLMEM``` analysis as a Python script, move to the ```tutorials/analysis/differential_gene_expression/``` directory.
  In ```dge_config.yaml```, update the ```dge_type``` and ```psedobulk/lmem_params```. ```dirpath``` and ```full_datapath``` represent the paths to save the results and the AnnData file, respectively.

- ```dge_config.yaml``` for ```DgePseudoBulk/DgeLMEM```

     ![Alt text](tutorial_config.png)


- Run either of the commands below in the terminal as per the ```dge_type``` and ```parameters```:
    - ```nohup /usr/bin/time --verbose python -u dge_pseudobulk_main.py --config dge_config.yaml >nohup_dge_pbk 2>&1 &``` 
    - ```nohup /usr/bin/time --verbose python -u dge_lmem_main.py --config dge_config.yaml >nohup_dge_lmem 2>&1 &```

## <span style="color: steelblue;">Comparison of identified up- and down- regulated genes by Pseudobulk and LMEM approaches</span>

In [ ]:
# Pseudobulk result for 'non-classical monocyte'
pbk_result_ncm.head()

In [ ]:
#LMEM result for 'non-classical monocyte'
lmem_result_ncm.head()

In [ ]:
#Adding column with absolute values for 'log2FoldChange' and 'coef_COVID-19' in the dataframes 
pbk_result_ncm['abs_log2FoldChange'] = np.abs(pbk_result_ncm['log2FoldChange'])
lmem_result_ncm['abs_coef_COVID-19'] = np.abs(lmem_result_ncm['coef_COVID-19'])

In [ ]:
pbk_result_ncm.head()

In [ ]:
lmem_result_ncm.head()

In [ ]:
#Setting required parameters to filter the up and down-regulated genes
fold_change = 1.5
log2_fc = np.log2(fold_change)
p_val = 0.05
coef_threshold = 0

In [ ]:
#Getting stats for up and down regulated genes in Pseudobulk result
pbk_up_reg_gene_df = pbk_result_ncm.loc[(pbk_result_ncm['log2FoldChange']>=log2_fc)&(pbk_result_ncm['pvalue']<=p_val)]

pbk_down_reg_gene_df = pbk_result_ncm.loc[(pbk_result_ncm['log2FoldChange']<=(-log2_fc))&(pbk_result_ncm['pvalue']<=p_val)]

In [ ]:
pbk_up_reg_gene_df.head()

In [ ]:
pbk_down_reg_gene_df.head()

In [ ]:
#Getting stats for up and down regulated genes in LMEM result
lmem_up_reg_gene_df = lmem_result_ncm.loc[(lmem_result_ncm['coef_COVID-19']>coef_threshold)&(lmem_result_ncm['pval_COVID-19']<=p_val)]

lmem_down_reg_gene_df = lmem_result_ncm.loc[(lmem_result_ncm['coef_COVID-19']<(-coef_threshold))&(lmem_result_ncm['pval_COVID-19']<=p_val)]

In [ ]:
lmem_up_reg_gene_df.head()

In [ ]:
lmem_down_reg_gene_df.head()

### <span style="color: steelblue;">Common up-regulated genes identified Pseudobulk v/s LMEM approaches</span>

In [ ]:
venn2([set(pbk_up_reg_gene_df['gene']), set(lmem_up_reg_gene_df['gene'])], set_labels=('Pseudobulk_up', 'LMEM_up'))

### <span style="color: steelblue;">Common down-regulated genes identified Pseudobulk v/s LMEM approaches</span>

In [ ]:
venn2([set(pbk_down_reg_gene_df['gene']), set(lmem_down_reg_gene_df['gene'])], set_labels=('Pseudobulk_down', 'LMEM_down'))